In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
import polars as pl
%load_ext memory_profiler
%load_ext line_profiler
%load_ext memory_magics

In [ ]:
# data ingest and sort
trade = ds.dataset("~/data/trade_taq2019.arrow", format = "arrow")
quote = ds.dataset("~/data/quote_taq2019.arrow", format = "arrow")

# construct scanner to database
#trade = ds.Scanner.from_dataset(trade)
#quote = ds.Scanner.from_dataset(quote)

# connect duckdb to database
con = duckdb.connect()

In [ ]:
con.query("SELECT * FROM trade WHERE Symbol = 'AAPL' AND Sale_Condition LIKE '%O%'").df()

In [ ]:
con.query("SELECT DATE_TRUNC('hour',Time) AS one_min, SUM(Trade_Volume), Symbol FROM trade GROUP BY one_min, Symbol ORDER BY one_min").df()

In [ ]:
con.query("SELECT Symbol,Exchange,AVG(Trade_Price),AVG(Trade_Volume) FROM trade GROUP BY Symbol,Exchange ORDER BY AVG(Trade_Price) DESC").df()

In [ ]:
con.query("SELECT DATE_TRUNC('hour', Time) AS hour, Symbol, AVG(0.5*(Offer_Price+Bid_Price)) AS mid FROM quote GROUP BY Symbol, hour ORDER BY hour").df()